In [46]:
import pandas as pd
import yfinance as yf
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import datetime as dt


Data Preparation: You'll need to download the stock price data from Yahoo Finance, clean and pre-process the data so that it can be used for training and testing.

Model Definition: Define the architecture of your model using the torch.nn library. You can use fully connected layers, recurrent layers, or convolutional layers, depending on the nature of your data.

Loss Function: Choose a loss function that measures how well your model is performing. Common loss functions for regression tasks include mean squared error (MSE) and mean absolute error (MAE).

Optimizer: Choose an optimizer that will adjust the parameters of your model during training. Common optimizers include Stochastic Gradient Descent (SGD), Adam, and Adagrad.

Training: Train your model on the pre-processed data using a training loop. In each iteration of the loop, you'll feed a batch of data through your model, compute the loss, and update the model parameters using the optimizer.

Evaluation: Evaluate the performance of your model on the test data to see how well it generalizes to new data.

Forecasting: Use the trained model to make predictions on future stock prices.

In [53]:
class LSTMpredict(nn.Module):
    def __init__(self, n_hidden=51):
        super(LSTMpredict, self).__init__()
        self.n_hidden = n_hidden
        # lstm1, lstm2, linear
        self.lstm1 = nn.LSTM(1, self.n_hidden)
        self.lstm2 = nn.LSTM(self.n_hidden, self.n_hidden)
        self.linear = nn.Linear(self.n_hidden, 1)

    def get_data(self):
        data = yf.download(self.ticker, period='8y', interval='1d')
        data = data.reset_index()
        data = data.rename(columns={'Date': 'ds', 'Adj Close': 'y'})
        df = pd.DataFrame(data)
        return df
    
    def forward(self, x, future=0):
        outputs = []
        n_samples = x.size(0)

        # initialize hidden states
        h_t = torch.zeros(1, n_samples, self.n_hidden, dtype=torch.double)
        c_t = torch.zeros(1, n_samples, self.n_hidden, dtype=torch.double)
        h_t2 = torch.zeros(1, n_samples, self.n_hidden, dtype=torch.double)
        c_t2 = torch.zeros(1, n_samples, self.n_hidden, dtype=torch.double)

        for input_t in x.split(1,dim=1):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]

        # predict the future using the last output
        for i in range(future):
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]

        outputs = torch.cat(outputs, dim=1)
        return outputs

In [54]:
lstm = LSTMpredict()
lstm.ticker = 'AAPL'
y = lstm.get_data()

[*********************100%***********************]  1 of 1 completed


In [55]:

train_input = torch.from_numpy(y[3:,:-1]).double()
train_target = torch.from_numpy(y[3:,1:]).double()
test_input = torch.from_numpy(y[:3,:-1]).double()
test_target = torch.from_numpy(y[:3,1:]).double()

model = LSTMpredict()
criterion = nn.MSELoss()

optimizer = optim.LBFGS(model.parameters(), lr=0.8)
n_steps = 15

for i in range(n_steps):
    print('STEP: ', i)

    def closure():
        optimizer.zero_grad()
        out = model(train_input)
        loss = criterion(out, train_target)
        print('loss:', loss.item())
        loss.backward()
        return loss


InvalidIndexError: (slice(3, None, None), slice(None, -1, None))